In [ ]:
# | echo: false
# | output: false

import sys
import os

from bs4 import BeautifulSoup

sys.path.append(os.path.abspath("../../"))

%reload_ext autoreload
%autoreload 2

import asyncio
from typing import Any, Callable, Coroutine, Literal
from python_utils.get_browser import get_browser_page_async

async def resolve_linkedin_username(uri: str) -> bool:
    page, close = await get_browser_page_async()
    response = None
    async def capture_response(resp):
        nonlocal response
        if uri in resp.url:
            response = resp
    page.on("response", capture_response)
    await page.goto("https://www.linkedin.com/")
    await page.evaluate(f"""
        fetch("{uri}", {{ "mode": "no-cors", "credentials": "include" }})
    """)
    await close()
    return response.ok

def resolve_instagram_username(username: str):
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto("https://www.instagram.com/")
        html_content: str = await page.evaluate(f"""
            fetch("{uri}", {{ "mode": "no-cors", "credentials": "include" }})
            .then(resp => resp.text(), resp => resp.text())
        """)
        await close()
        title = BeautifulSoup(html_content, "html.parser").find('title').text
        return username.lower() in title.lower()
    return resolve

def resolve_username_from_title(username: str):
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri, wait_until='load')
        await page.wait_for_timeout(300)
        title: str = await page.title()
        await close()
        return username.lower() in title.lower()
    return resolve

def resolve_document_contains(search: str, evaluate_str: str) -> bool:
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri)
        inner_text = await page.evaluate(evaluate_str)
        await close()
        return search.lower() in inner_text.lower()
    return resolve

class UsernameChecker:
    def __init__(self, username):
        self.username = username
        self.platforms: dict[Literal["name", "uri", "resolve", "message"], Any]= [
            {
                "name": "LinkedIn Company Page",
                # can replace "www." with "de.", ".ke", ".ug", etc
                "uri": f"https://www.linkedin.com/company/{username}",
                "resolve": resolve_linkedin_username,
            },
            {
                "name": "LinkedIn User Account",
                # can replace "www." with "de.", ".ke", ".ug", etc
                "uri": f"https://www.linkedin.com/in/{username}",
                "resolve": resolve_linkedin_username,
                # https://github.com/tomquirk/linkedin-api
                "message": (
                    "❌ Taken or Private."
                    " 😶 LinkedIn private user profiles are not easy to check,"
                    " login into LinkedIn and go to"
                    f' "https://www.linkedin.com/in/{username}"')
            },
            {
                "name": "Instagram Account (Personal/Business)",
                # trailing / is important here
                "uri": f"https://www.instagram.com/{username}/",
                "resolve": resolve_instagram_username(username),
            },
            {
                "name": "Youtube Channel",
                "uri": f"https://www.youtube.com/@{username}",
                "resolve": resolve_document_contains(f"@{username}", "document.body.outerHTML"),
            },
            {
                "name": "Tiktok Account",
                "uri": f"https://www.tiktok.com/@{username}",
                "resolve": resolve_username_from_title(username),
            },
            {
                "name": "X/Twitter Account",
                "uri": f"https://x.com/{username}",
                not accurate!
                "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            },
        ]

    async def check_async(self):
        tasks = [self._check_async(**platform) for platform in self.platforms]
        return await asyncio.gather(*tasks)

    async def _check_async(
            self, *, 
            name: str, uri: str, resolve: Callable[[str], Coroutine[Any, Any, bool]], message: str = None):
        try:
            is_available = await resolve(uri)
            if is_available == True:
                return {
                    "platform": name,
                    'available': False,
                    'message': f"{self.username}: ❌ Taken",
                    'url': uri
                }
            if message:
                return {
                    "platform": name,
                    'available': True,
                    'message': message,
                    'url': uri
                }
            else:
                return {
                    "platform": name,
                    'available': True,
                    'message': f"{self.username}: ✅ Available",
                    'url': uri
                }
        except Exception as e:
            return {
                "platform": name,
                'available': None,
                'error': str(e)
            }

In [23]:
# linkedin private user => kamau
# Create checker instance
checker = UsernameChecker("One_MorePlace")
# Check availability
await checker.check_async()

[{'platform': 'LinkedIn Company Page',
  'available': True,
  'message': 'One_MorePlace: ✅ Available',
  'url': 'https://www.linkedin.com/company/One_MorePlace'},
 {'platform': 'LinkedIn User Account',
  'available': True,
  'message': '❌ Taken or Private. 😶 LinkedIn private user profiles are not easy to check, login into LinkedIn and go to "https://www.linkedin.com/in/One_MorePlace"',
  'url': 'https://www.linkedin.com/in/One_MorePlace'},
 {'platform': 'Instagram Account (Personal/Business)',
  'available': False,
  'message': 'One_MorePlace: ❌ Taken',
  'url': 'https://www.instagram.com/One_MorePlace/'},
 {'platform': 'Youtube Channel',
  'available': False,
  'message': 'One_MorePlace: ❌ Taken',
  'url': 'https://www.youtube.com/@One_MorePlace'},
 {'platform': 'Tiktok Account',
  'available': True,
  'message': 'One_MorePlace: ✅ Available',
  'url': 'https://www.tiktok.com/@One_MorePlace'},
 {'platform': 'X/Twitter Account',
  'available': True,
  'message': 'One_MorePlace: ✅ Availa